## Problema 2
Estudiantes:
- Matías Fuentes
- Larry Uribne

In [ ]:
!pip install pyspark

In [ ]:
!wget https://github.com/IIC2440/Syllabus-2023-1/raw/main/Actividades/cora.zip
!unzip /content/cora.zip

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import math
spark = SparkSession.builder \
    .getOrCreate()
sc = spark.sparkContext

Recibir input de juguete

In [ ]:
nodes = list(range(1, 9))
edges = [(1,2,4), (1,3,2), (1,4,1), (3,2,1), (3,5,6),(4,5,9),(4,6,2),(7,8,9), (5,6,1)]

Generar RDDs

In [ ]:
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)

Formateamos las aristas

In [ ]:
rdd_edges = rdd_edges.map(lambda x: (x[0], (x[1], x[2])))
rdd_edges.collect()

1. Escoge el nodo inicial, este nodo tiene costo acumulado 0 y todos los demás tienen costo acumulado
infinito.

In [ ]:
def init_rdd(rdd_nodes, initial_node):
    bc_initial_node = sc.broadcast(initial_node) # Disponibilizar para todos los worker
    return rdd_nodes.map(lambda x: (x, math.inf) if x != bc_initial_node.value else (x, 0))
rdd_init = init_rdd(rdd_nodes, 1)
rdd_init.collect()

2. En cada iteración, cada nodo comunica el costo acumulado a sus vecinos. Cada nodo recibe este costo,
sumado con el costo de atravesar la arista.

In [ ]:
rdd_edges.join(rdd_init).collect()

In [ ]:
def pass_msg(rdd_edges, rdd_prev):
    return rdd_edges.join(rdd_prev).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()
rdd_pass_msg = pass_msg(rdd_edges, rdd_init)
rdd_pass_msg.collect()

3. Para hacer merge de todos los mensajes dejamos el mínimo de todos los costos. Así, actualizamos cada
nodo con el costo mínimo recibido solo si es menor al costo acumulado que ya tenía ese nodo.

In [ ]:
def reduce_msg(rdd_pass_msg):
    return rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))
    
rdd_reduce_msg = reduce_msg(rdd_pass_msg)
rdd_reduce_msg.collect()

In [ ]:
def robust_min(x):
    a, b = x
    try:
        return min(a,b)
    except:
        return a if a != None else b
    
def update_cost(rdd_prev, rdd_reduce_msg):
    return rdd_prev.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))

rdd_updated_cost = update_cost(rdd_init, rdd_reduce_msg)
rdd_updated_cost.collect()

4. Si en dos iteraciones el costo en llegar para cada nodo no cambia, entonces nos detenemos.

In [ ]:
def stop_condition(rdd_pre, rdd_post):
    # RDD donde las llaves son los nodos y los valores son 1 si no hubo cambio y 0 si lo hubo
    rdd_track_changes = rdd_pre.join(rdd_post).mapValues(lambda x: int(x[0] == x[1]))
    # Intersección
    n_changes = rdd_track_changes.values().reduce(lambda x, y: x + y) 
    return n_changes == 0
stop_condition(rdd_init, rdd_updated_cost)

Consolidar en una función

In [157]:
def init_rdd(rdd_nodes, initial_node):
    bc_initial_node = sc.broadcast(initial_node) # Disponibilizar para todos los worker
    return rdd_nodes.map(lambda x: (x, math.inf) if x != bc_initial_node.value else (x, 0))

def pass_msg(rdd_edges, rdd_prev):
    return rdd_edges.join(rdd_prev).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()

def reduce_msg(rdd_pass_msg):
    return rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))

def robust_min(x):
    a, b = x
    try:
        return min(a,b)
    except:
        return a if a != None else b
    
def update_cost(rdd_prev, rdd_reduce_msg):
    return rdd_prev.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))
    
def stop_condition(rdd_pre, rdd_post):
    # RDD donde las llaves son los nodos y los valores son 0 si no hubo cambio y 1 si lo hubo
    rdd_track_changes = rdd_pre.join(rdd_post).mapValues(lambda x: int(x[0] != x[1]))
    
    # Intersección
    n_changes = rdd_track_changes.values().reduce(lambda x, y: x + y) 
    return n_changes == 0

def single_source_shortest_path(rdd_nodes, rdd_edges, initial_node, max_iterations = 10000):
    rdd_edges = rdd_edges.map(lambda x: (x[0], (x[1], x[2])))

    # Nodo inicial en 0, el resto en infinito
    rdd_init = init_rdd(rdd_nodes, initial_node)
    rdd_prev_cost = rdd_init
    
    stop_counter = 0
    for i in range(max_iterations):
        rdd_pass_msg = pass_msg(rdd_edges, rdd_prev_cost)
        #rdd_pass_msg = rdd_edges.join(rdd_prev_cost).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()
        rdd_reduce_msg = reduce_msg(rdd_pass_msg)
        # rdd_reduce_msg = rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))
        rdd_updated_cost = update_cost(rdd_prev_cost, rdd_reduce_msg)
        #rdd_updated_cost = rdd_prev_cost.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))
        if stop_condition(rdd_prev_cost, rdd_updated_cost):
            stop_counter += 1
            if stop_counter == 2:
                break
        else:
            stop_counter = 0 
        rdd_prev_cost = rdd_updated_cost
        #print("Iteracion:", i)
        #print(rdd_updated_cost.collect())
    print(f'Total iterations: {i+1}')
    return rdd_updated_cost


Ejecutar y mostrar resultados

In [158]:
nodes = list(range(1, 9))
edges = [(1,2,4), (1,3,2), (1,4,1), (3,2,1), (3,5,6),(4,5,9),(4,6,2),(7,8,9), (5,6,1)]

initial_node = 1
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)
rdd_result = single_source_shortest_path(rdd_nodes, rdd_edges, initial_node)
rdd_result.collect()

### Pruebas con Cora

Cargar datos de Cora y formatear para la tarea

In [ ]:
citas = pd.read_csv('cora/cora.cites',sep="\t",
                    header=None,
                    names=["target", "source"])
nodes = list(set(list(citas.target.values) + list(citas.source.values)))
edges = []
for target, source in citas.values:
    edges.append((source, target, 1))

Ejecutar con un nodo que es miembro de la componente conectada más grande, y tiene 5 vértices de salida

In [154]:
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)
initial_node = 1105932
rdd_result = single_source_shortest_path(rdd_nodes, rdd_edges, initial_node)